<a href="https://colab.research.google.com/github/ZijingZhang-0114/mgmt467-analytics-portfolio/blob/main/Lab2_Advanced_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab: Vertex AI–Assisted BigQuery Analytics — Example Prompts
**Goal:** Practice moving from simple SQL to complex analytics in BigQuery using *only* carefully engineered prompts with Vertex AI (Gemini).  
**Important:** This notebook contains **prompts only** (no starter code). Paste the prompts into **Vertex AI Studio**, **Vertex AI in Colab Enterprise**, or your chosen chat interface, and then run the generated SQL directly in **BigQuery**. If you decide to automate later, you can ask Vertex AI to convert the winning SQL into a Colab pipeline.

## How to use this prompts-only notebook
1. Open **Vertex AI Studio** (or Gemini in Colab Enterprise chat panel).  
2. Copy a prompt from this notebook and paste it into the model. Do **not** paste any code from here; let the model generate it.  
3. Run the generated SQL in **BigQuery** (Console → BigQuery Studio).  
4. Iterate: refine the prompt when results aren’t what you expect.  
5. Document: capture your final SQL, plus a one-sentence takeaway, in your notes/README.

## Dataset assumptions
Use one of these sources (adjust table paths accordingly):
- **Global Superstore (Kaggle)** loaded into BigQuery (e.g., `[YOUR_PROJECT].superstore_data.sales`)  
- **TheLook eCommerce** public dataset: `bigquery-public-data.thelook_ecommerce`  
If you are using *Global Superstore*, make sure column names match your schema (e.g., `Order_Date`, `Region`, `Category`, `Sub_Category`, `Sales`, `Profit`, `Discount`, `State`, `Customer_ID`, `Ship_Mode`).

---
## Prompting guardrails (quick checklist)
- **Be explicit**: table path, column names, filters, output columns, sort order, and limits.  
- **Ask for runnable SQL**: “Return a BigQuery SQL block only.”  
- **Control cost**: ask for `LIMIT` during exploration and remove it for the final run.  
- **Validate**: request a brief explanation of why each clause is present and how you can sanity-check results.
---

## Install Dependencies

In [ ]:
# Install the Google Cloud BigQuery client library
!pip install google-cloud-bigquery==3.17.0 pandas==2.1.4


In [ ]:
# Authenticate your Colab environment
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Copy Schema to a dataframe

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Replace with your Google Cloud Project ID
project_id = 'mgmt-46700-471119' # This is derived from your provided table name
dataset_id = 'superstore_data'
table_id = 'sale'

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

# Get the table object
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

# Extract schema information
schema_list = []
for field in table.schema:
    schema_list.append({
        'name': field.name,
        'field_type': field.field_type,
        'mode': field.mode,
        'description': field.description
    })

# Convert to Pandas DataFrame
schema_df = pd.DataFrame(schema_list)

# Display the schema DataFrame (optional, for verification)
print("Schema DataFrame created:")
# To see the output, run the code.


Schema DataFrame created:


## CLean Column Names

In [ ]:
# --- 1. Clean the Column Names ---
# Create a 'clean_name' column with standard naming conventions:
# lowercase, with spaces and hyphens replaced by underscores.
schema_df['clean_name'] = schema_df['name'].str.lower().str.replace(' ', '_').str.replace('-', '_')


# --- 2. Generate the Aliases for the SELECT Clause ---
column_expressions = []
for index, row in schema_df.iterrows():
    original_name = row['name']
    clean_name = row['clean_name']

    # If the original name contains a space or special character, it needs to be
    # enclosed in backticks (`) in the SQL statement.
    if ' ' in original_name or '-' in original_name:
        expression = f'`{original_name}` AS {clean_name}'
    else:
        # If the name is already clean, we still alias it for consistency.
        expression = f'{original_name} AS {clean_name}'
    column_expressions.append(expression)

# Join all the individual column expressions into a single, formatted string.
select_clause = ",\n  ".join(column_expressions)


# --- 3. Construct the Final CREATE VIEW Statement ---
new_view_id = 'superstore_clean' # You can change this if you like

create_view_sql = f"""
CREATE OR REPLACE VIEW `{project_id}.{dataset_id}.{new_view_id}` AS
SELECT
  {select_clause}
FROM
  `{project_id}.{dataset_id}.{table_id}`;
"""

# --- 4. Print the Final SQL ---
print("--- Copy the SQL below and run it in your BigQuery Console ---")
print(create_view_sql)

--- Copy the SQL below and run it in your BigQuery Console ---

CREATE OR REPLACE VIEW `mgmt-46700-471119.superstore_data.superstore_clean` AS
SELECT
  `Row ID` AS row_id,
  `Order ID` AS order_id,
  `Order Date` AS order_date,
  `Ship Date` AS ship_date,
  `Ship Mode` AS ship_mode,
  `Customer ID` AS customer_id,
  `Customer Name` AS customer_name,
  Segment AS segment,
  City AS city,
  State AS state,
  Country AS country,
  `Postal Code` AS postal_code,
  Market AS market,
  Region AS region,
  `Product ID` AS product_id,
  Category AS category,
  `Sub-Category` AS sub_category,
  `Product Name` AS product_name,
  Sales AS sales,
  Quantity AS quantity,
  Discount AS discount,
  Profit AS profit,
  `Shipping Cost` AS shipping_cost,
  `Order Priority` AS order_priority
FROM
  `mgmt-46700-471119.superstore_data.sale`;



## Generate View with standard column naming convention

In [ ]:
# Execute the CREATE VIEW SQL query
try:
    query_job = client.query(create_view_sql)  # API request
    query_job.result()  # Waits for the query to finish
    print(f"View '{new_view_id}' created/replaced successfully in dataset '{dataset_id}'.")
except Exception as e:
    print(f"An error occurred while creating the view: {e}")

# Now, let's print 10 rows from the newly created view to verify
print(f"\n--- First 10 rows from the new view '{new_view_id}' ---")
try:
    # Construct a reference to the new view
    view_table_ref = client.dataset(dataset_id).table(new_view_id)

    # Fetch the first 10 rows
    rows = client.list_rows(view_table_ref, max_results=10)

    # Print header
    print(" | ".join([field.name for field in rows.schema]))
    print("-" * 80) # Separator

    # Print rows
    for row in rows:
        print(" | ".join([str(item) for item in row.values()]))

except Exception as e:
    print(f"An error occurred while fetching rows from the view: {e}")



View 'superstore_clean' created/replaced successfully in dataset 'superstore_data'.

--- First 10 rows from the new view 'superstore_clean' ---
row_id | order_id | order_date | ship_date | ship_mode | customer_id | customer_name | segment | city | state | country | postal_code | market | region | product_id | category | sub_category | product_name | sales | quantity | discount | profit | shipping_cost | order_priority
--------------------------------------------------------------------------------
An error occurred while fetching rows from the view: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/mgmt-46700-471119/datasets/superstore_data/tables/superstore_clean/data?maxResults=10&formatOptions.useInt64Timestamp=True&prettyPrint=false: Cannot list a table of type VIEW.


In [ ]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  order_id,
  customer_name,
  product_name,
  sales,
  profit
FROM
  `mgmt-467-47888.lab1_foundation.superstore_clean`
LIMIT 10;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...

❌ An error occurred: 403 Access Denied: Table mgmt-467-47888:lab1_foundation.superstore_clean: User does not have permission to query table mgmt-467-47888:lab1_foundation.superstore_clean, or perhaps it does not exist.; reason: accessDenied, message: Access Denied: Table mgmt-467-47888:lab1_foundation.superstore_clean: User does not have permission to query table mgmt-467-47888:lab1_foundation.superstore_clean, or perhaps it does not exist.

Location: US
Job ID: 5030da35-aa49-4998-bae1-9127966329ff



## Part A — SQL Warm‑Up (SELECT, WHERE, ORDER BY, LIMIT, DISTINCT)
**Aim:** Build confidence with precise, unambiguous prompts that yield clean, runnable SQL.

### A1. Unique values (DISTINCT)
**Prompt (paste in Vertex AI):**
```
Act as a senior BigQuery analyst. Produce a **single runnable BigQuery SQL** (no commentary) for:
- Task: List all unique `Sub_Category` values sold in the 'West' region.
- Table: `mgmt-467-47888.lab1_foundation.superstore`
- Filter: `Region = 'West'`
- Output: a single column named `Sub_Category`
- Sort: alphabetically A→Z
- Add: `LIMIT 100` to control cost during exploration.
```
**Reflection:** Did the result match your expectations? If not, what ambiguity in your prompt might have caused the mismatch?

In [ ]:
query_string = """
SELECT
    DISTINCT sub_category
FROM
    `mgmt-46700-471119.superstore_data.superstore_clean`
WHERE
    region = 'West'
ORDER BY
    sub_category ASC
LIMIT 100
"""

try:
    # Execute the query
    query_job = client.query(query_string)

    # Wait for the job to complete and get the results as a DataFrame
    results_df = query_job.to_dataframe()

    # Display the results
    print(f"Query successful. Found {len(results_df)} unique sub-categories in the 'West' region.")
    display(results_df)

except Exception as e:
    print(f"An error occurred: {e}")

Query successful. Found 17 unique sub-categories in the 'West' region.


,sub_category
0,Accessories
1,Appliances
2,Art
3,Binders
4,Bookcases
5,Chairs
6,Copiers
7,Envelopes
8,Fasteners
9,Furnishings


### A2. Top‑N by metric (ORDER BY … DESC)
**Prompt:**
```
BigQuery SQL only.
Task: Return the top 10 customers by total profit.
Table: `mgmt-467-47888.lab_foundation.superstore`
Columns used: `Customer_ID`, `Profit`
Output columns: `Customer_ID`, `total_profit`
Logic: SUM Profit per customer, order by `total_profit` DESC
Add `LIMIT 10`.
```
**Tip:** If your schema uses different identifiers (e.g., `Customer Name`), restate column names explicitly.

In [ ]:
query_string = """
SELECT
  customer_id,
  SUM(profit) AS total_profit
FROM
  `mgmt-46700-471119.superstore_data.superstore_clean`
GROUP BY
  customer_id
ORDER BY
  total_profit DESC
LIMIT 10
"""

try:
    # Execute the query
    query_job = client.query(query_string)

    # Wait for the job to complete and get the results as a DataFrame
    results_df = query_job.to_dataframe()

    # Display the results
    print(f"Query successful. Found top {len(results_df)} customers by profit.")
    display(results_df)

except Exception as e:
    print(f"An error occurred: {e}")

Query successful. Found top 10 customers by profit.


,customer_id,total_profit
0,TC-20980,8787.47490
1,RB-19360,8523.95150
2,SC-20095,8106.21790
3,BE-11335,7790.69630
4,HL-15040,7657.50178
5,AB-10105,6912.61450
6,SP-20920,6649.62626
7,HM-14860,6544.88640
8,TA-21385,6274.98910
9,SE-20110,5863.62392


### A3. Basic filtering (WHERE) + sanity checks
**Prompt:**
```
BigQuery SQL only.
Task: Count orders shipped with each `Ship_Mode`, but only for orders in the 'Technology' category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Ship_Mode`, `order_count`
Logic: COUNT(*) grouped by `Ship_Mode`
Sort by `order_count` DESC
```
**Validation ask:** “Also list two quick sanity checks to verify the numbers.”

In [ ]:
query_string = """
SELECT
  ship_mode,
  COUNT(*) AS order_count
FROM
  `mgmt-46700-471119.superstore_data.superstore_clean`
WHERE
  category = 'Technology'
GROUP BY
  ship_mode
ORDER BY
  order_count DESC
"""

try:
    # Execute the query
    query_job = client.query(query_string)

    # Wait for the job to complete and get the results as a DataFrame
    results_df = query_job.to_dataframe()

    # Display the results
    print(f"Query successful. Found {len(results_df)} ship modes for the 'Technology' category.")
    display(results_df)

except Exception as e:
    print(f"An error occurred: {e}")

Query successful. Found 4 ship modes for the 'Technology' category.


,ship_mode,order_count
0,Standard Class,6117
1,Second Class,2030
2,First Class,1476
3,Same Day,518


## Part B — Grouped Analytics (GROUP BY, HAVING)
**Aim:** Turn raw facts into grouped metrics and filtered aggregations.

### B1. KPI aggregation with WHERE + GROUP BY
**Prompt:**
```
BigQuery SQL only.
Task: Compute monthly revenue for the last 12 full months.
Table: `[YOUR_PROJECT].superstore_data.sales`
Assume: `Order_Date` is a DATE or TIMESTAMP column named exactly `Order_Date`.
Output: `year_month` (YYYY-MM format), `monthly_revenue`
Logic: Truncate date to month, SUM `Sales`, filter to last 12 full months.
Sort by `year_month` ascending.
Include a `LIMIT` safeguard for exploration.
```

In [ ]:
query_string = """
SELECT
  FORMAT_DATE('%Y-%m', DATE_TRUNC(order_date, MONTH)) AS year_month,
  ROUND(SUM(sales), 2) AS monthly_revenue
FROM
  `mgmt-46700-471119.superstore_data.superstore_clean`
GROUP BY
  year_month
ORDER BY
  year_month ASC
LIMIT 15 -- Safeguard for exploration
"""

try:
    # Execute the query
    query_job = client.query(query_string)

    # Wait for the job to complete and get the results as a DataFrame
    results_df = query_job.to_dataframe()

    # Display the results
    print(f"Query successful. Found {len(results_df)} months of revenue data.")
    display(results_df)

except Exception as e:
    print(f"An error occurred: {e}")

Query successful. Found 15 months of revenue data.


,year_month,monthly_revenue
0,2011-01,98898.49
1,2011-02,91152.16
2,2011-03,145729.37
3,2011-04,116915.76
4,2011-05,146747.84
5,2011-06,215207.38
6,2011-07,115510.42
7,2011-08,207581.49
8,2011-09,290214.46
9,2011-10,199071.26


### B2. Post‑aggregation filter (HAVING)
**Prompt:**
```
BigQuery SQL only.
Task: Find sub-categories whose total profit over the entire dataset is negative.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Sub_Category`, `total_profit`
Logic: SUM `Profit` GROUP BY `Sub_Category`, HAVING SUM(Profit) < 0
Sort by `total_profit` ASC (most negative first).
```
**Why HAVING?** Ask the model to include a 1-sentence explanation of why HAVING is used instead of WHERE here.

In [ ]:
query_string = """
SELECT
  sub_category,
  ROUND(SUM(profit), 2) AS total_profit
FROM
  `mgmt-46700-471119.superstore_data.superstore_clean`
GROUP BY
  sub_category
HAVING
  SUM(profit) < 0
ORDER BY
  total_profit ASC
"""

try:
    # Execute the query
    query_job = client.query(query_string)

    # Wait for the job to complete and get the results as a DataFrame
    results_df = query_job.to_dataframe()

    # Display the results
    print(f"Query successful. Found {len(results_df)} sub-categories with negative profit.")
    display(results_df)

except Exception as e:
    print(f"An error occurred: {e}")

Query successful. Found 1 sub-categories with negative profit.


,sub_category,total_profit
0,Tables,-64083.39


## Part C — Joins (dimension enrichment)
**Aim:** Use joins to enhance facts with attributes.

### C1. Join facts to a small dimension
*(If you have a customer or product dimension in your schema, use it. Otherwise, request a synthetic example.)*  
**Prompt:**
```
BigQuery SQL only.
Task: Join the sales table to a product dimension to report `Product_ID`, `Product_Name`, and total sales.
Tables: `[YOUR_PROJECT].superstore_data.sales` as s, `[YOUR_PROJECT].superstore_data.products` as p
Join key: `s.Product_ID = p.Product_ID`
Output: `Product_ID`, `Product_Name`, `total_sales`
Sort by `total_sales` DESC
```
**If you lack a dimension table:** Ask the model how to simulate one temporarily via a CTE.

In [ ]:
query_string = """
SELECT
  product_id,
  product_name,
  ROUND(SUM(sales), 2) AS total_sales
FROM
  `mgmt-46700-471119.superstore_data.superstore_clean`
GROUP BY
  product_id,
  product_name
ORDER BY
  total_sales DESC
LIMIT 10 -- Show top 10 products
"""

try:
    # Execute the query
    query_job = client.query(query_string)

    # Wait for the job to complete and get the results as a DataFrame
    results_df = query_job.to_dataframe()

    # Display the results
    print(f"Query successful. Found top {len(results_df)} products by sales.")
    display(results_df)

except Exception as e:
    print(f"An error occurred: {e}")

Query successful. Found top 10 products by sales.


,product_id,product_name,total_sales
0,TEC-CO-10004722,Canon imageCLASS 2200 Advanced Copier,61599.82
1,TEC-PH-10004664,"Nokia Smart Phone, with Caller ID",30041.55
2,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Bin...,27453.38
3,TEC-MA-10002412,Cisco TelePresence System EX90 Videoconferenci...,22638.48
4,TEC-PH-10004823,"Nokia Smart Phone, Full Size",22262.10
5,FUR-CH-10002024,HON 5400 Series Task Chairs for Big and Tall,21870.58
6,FUR-CH-10000027,"SAFCO Executive Leather Armchair, Black",21329.73
7,OFF-AP-10004512,"Hoover Stove, Red",21147.08
8,OFF-BI-10001359,GBC DocuBind TL300 Electric Binding System,19823.48
9,OFF-BI-10000545,GBC Ibimaster 500 Manual ProClick Binding System,19024.50


## Part D — Common Table Expressions (CTEs)
**Aim:** Make complex logic readable and testable in steps.

### D1. Multi‑step ranking with CTEs
**Prompt:**
```
BigQuery SQL only.
Goal: Within each `Region`, rank states by total sales and return top 3 per region.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE 1 (`state_sales`): SUM(Sales) by `Region`, `State`
CTE 2 (`ranked_state_sales`): Add `RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as `sales_rank`
Final SELECT: rows where `sales_rank <= 3`
Output columns: `Region`, `State`, `total_sales`, `sales_rank`
Sort: by `Region`, then `sales_rank`
```
**Ask for**: a one-paragraph explanation of each step, then **provide only the final runnable SQL**.

# Task
Using the BigQuery table `\"mgmt-46700-471119.superstore_data.superstore_clean\"`, find the top 3 states by total sales within each region. The query should use Common Table Expressions (CTEs) and a window function to rank the states. The final output should include the region, state, total sales, and the sales rank, sorted by region and then by rank.

## Generate Multi-Step Ranking Query

### Subtask:
Construct a BigQuery SQL query using Common Table Expressions (CTEs) and a window function to rank states by sales within each region.


**Reasoning**:
The user wants to build a complex BigQuery query in steps. This step involves creating a Python string variable `query_string` containing the SQL for two Common Table Expressions (CTEs) and a preliminary final `SELECT` statement, as per the detailed instructions. This sets up the query structure which will be completed and executed in a subsequent step.



In [ ]:
# This query is intentionally incomplete and will be finalized in the next step.
query_string = """
WITH
  state_sales AS (
    -- First, calculate the total sales for each state within each region
    SELECT
      region,
      state,
      SUM(sales) AS total_sales
    FROM
      `mgmt-46700-471119.superstore_data.superstore_clean`
    GROUP BY
      region,
      state
  ),
  ranked_state_sales AS (
    -- Second, rank the states within each region based on their total sales
    SELECT
      region,
      state,
      total_sales,
      RANK() OVER (PARTITION BY region ORDER BY total_sales DESC) AS sales_rank
    FROM
      state_sales
  )
-- The final selection will be completed in the next step
SELECT
  region,
  state,
  total_sales,
  sales_rank
FROM
  ranked_state_sales
"""

print("Partial query string defined. Ready for finalization in the next step.")

Partial query string defined. Ready for finalization in the next step.


In [ ]:
final_query_string = query_string + """
WHERE
  sales_rank <= 3
ORDER BY
  region,
  sales_rank;
"""

print("--- Final Query ---")
print(final_query_string)

print("\n--- Executing Query and Fetching Results ---")
try:
    # Execute the query
    query_job = client.query(final_query_string)

    # Wait for the job to complete and get the results as a DataFrame
    results_df = query_job.to_dataframe()

    # Display the results
    print(f"Query successful. Found top {len(results_df)} state rankings across all regions.")
    display(results_df)

except Exception as e:
    print(f"An error occurred: {e}")

--- Final Query ---

WITH
  state_sales AS (
    -- First, calculate the total sales for each state within each region
    SELECT
      region,
      state,
      SUM(sales) AS total_sales
    FROM
      `mgmt-46700-471119.superstore_data.superstore_clean`
    GROUP BY
      region,
      state
  ),
  ranked_state_sales AS (
    -- Second, rank the states within each region based on their total sales
    SELECT
      region,
      state,
      total_sales,
      RANK() OVER (PARTITION BY region ORDER BY total_sales DESC) AS sales_rank
    FROM
      state_sales
  )
-- The final selection will be completed in the next step
SELECT
  region,
  state,
  total_sales,
  sales_rank
FROM
  ranked_state_sales

WHERE
  sales_rank <= 3
ORDER BY
  region,
  sales_rank;


--- Executing Query and Fetching Results ---
Query successful. Found top 39 state rankings across all regions.


,region,state,total_sales,sales_rank
0,Africa,Gauteng,51604.32000,1
1,Africa,Kinshasa,42536.52000,2
2,Africa,Al Qahirah,38439.78000,3
3,Canada,Ontario,35450.40000,1
4,Canada,Quebec,10924.17000,2
5,Canada,British Columbia,9544.38000,3
6,Caribbean,Santo Domingo,78713.66344,1
7,Caribbean,Santiago de Cuba,32452.54776,2
8,Caribbean,Granma,17002.53732,3
9,Central,Ile-de-France,317822.54400,1


**Reasoning**:
The previous step defined the basic structure of the CTEs. This step finalizes the query by adding the `WHERE` clause to filter for the top 3 ranks and the `ORDER BY` clause to sort the results. It then executes the complete SQL query using the BigQuery client and displays the resulting DataFrame.



## Summary:

### Q&A
**Question:** What are the top 3 states by total sales within each region?

**Answer:** The analysis identified the top 3 states by sales for each of the 13 regions. For example:
*   In the **West** region, the top states are California (\\$457,687.63), Washington (\\$138,641.27), and Arizona (\\$35,282.00).
*   In the **East** region, the top states are New York (\\$310,876.27), Pennsylvania (\\$116,511.91), and Ohio (\\$78,258.14).
*   In the **North** region, the top states are England (\\$485,170.97), Distrito Federal (\\$128,422.91), and Jalisco (\\$43,291.45).

### Data Analysis Key Findings
*   A multi-step query using Common Table Expressions (CTEs) and the `RANK()` window function successfully ranked states by sales within each region.
*   The query returned a total of 39 results, corresponding to the top 3 states across 13 distinct geographical regions.
*   Sales performance is highly concentrated in specific states. For example, California in the 'West' region has total sales of \\$457,687.63, which is more than three times the sales of the second-ranked state, Washington (\\$138,641.27).
*   England emerged as a top-performing area in the 'North' region, with total sales of \\$485,170.97.

### Insights or Next Steps
*   **Insight:** The significant gap in sales between the top-ranked state and the others within the same region (e.g., California vs. Washington) suggests that market penetration and sales strategies are highly effective in certain key areas but less so in others.
*   **Next Step:** Investigate the factors driving the success of top-performing states like California and England. This could involve analyzing which product categories or customer segments are most profitable in these areas to identify strategies that could be replicated in underperforming states.


### D2. Time‑boxed “most improved” analysis
**Prompt:**
```
BigQuery SQL only.
Goal: Identify the top 5 sub-categories with the largest YoY revenue increase from 2023 to 2024.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `yr_sales`: SUM(Sales) by `Sub_Category` and `year` extracted from `Order_Date`
Final: pivot or self-join to compute delta (2024 minus 2023) as `yoy_delta`
Output: `Sub_Category`, `sales_2023`, `sales_2024`, `yoy_delta`
Order by `yoy_delta` DESC
Limit 5
```
**Validation:** Ask the model for two quick failure modes (e.g., missing years) and how to handle them.

# Task
Using the BigQuery table `"mgmt-46700-471119.superstore_data.superstore_clean"`, identify the top 5 sub-categories with the largest year-over-year (YoY) revenue increase between 2013 and 2014. The analysis should use a Common Table Expression (CTE) to calculate annual sales and then pivot the data to find the YoY delta. Display the top 5 results and provide a summary explaining how the query handles potential failure modes, such as missing data for certain years.

## Generate YoY Growth Query

### Subtask:
Create a BigQuery SQL query using a Common Table Expression (CTE) to calculate total sales per sub-category for the years 2013 and 2014. The query will then pivot this data to calculate the year-over-year (YoY) sales delta for each sub-category.


**Reasoning**:
The user wants to write a query to calculate the year-over-year sales growth. I will construct the SQL query using a CTE to get annual sales for 2013 and 2014, and then pivot the data to calculate the difference. This query will be executed in Python to find and display the top 5 sub-categories with the largest growth.



In [ ]:
# This query identifies the top 5 sub-categories with the largest YoY revenue increase from 2013 to 2014.
# The original prompt asked for 2023-2024, but the dataset is historical, so we use the last two full years available.
query_string = """
WITH
  yr_sales AS (
    -- First, calculate total sales per sub-category for the relevant years (2013 and 2014)
    SELECT
      sub_category,
      EXTRACT(YEAR FROM order_date) AS year,
      SUM(sales) AS annual_sales
    FROM
      `mgmt-46700-471119.superstore_data.superstore_clean`
    WHERE
      EXTRACT(YEAR FROM order_date) IN (2013, 2014)
    GROUP BY
      sub_category,
      year
  )
-- Pivot the data to create separate columns for 2013 and 2014 sales and calculate the difference
SELECT
  sub_category,
  ROUND(SUM(IF(year = 2013, annual_sales, 0)), 2) AS sales_2013,
  ROUND(SUM(IF(year = 2014, annual_sales, 0)), 2) AS sales_2014,
  -- Calculate the year-over-year change
  ROUND(SUM(IF(year = 2014, annual_sales, 0)) - SUM(IF(year = 2013, annual_sales, 0)), 2) AS yoy_delta
FROM
  yr_sales
GROUP BY
  sub_category
ORDER BY
  yoy_delta DESC
LIMIT 5;
"""

print("--- Executing YoY Growth Query ---")
try:
    # Execute the query
    query_job = client.query(query_string)

    # Wait for the job to complete and get the results as a DataFrame
    results_df = query_job.to_dataframe()

    # Display the results
    print(f"Query successful. Found top {len(results_df)} sub-categories by YoY growth.")
    display(results_df)

except Exception as e:
    print(f"An error occurred: {e}")

--- Executing YoY Growth Query ---
Query successful. Found top 5 sub-categories by YoY growth.


,sub_category,sales_2013,sales_2014,yoy_delta
0,Bookcases,376025.80,513196.71,137170.91
1,Copiers,415514.89,550384.89,134870.01
2,Appliances,254951.13,359786.78,104835.65
3,Phones,453519.00,552006.38,98487.38
4,Storage,309475.80,383427.29,73951.49


**Reasoning**:
The user has provided detailed instructions for a new step that involves creating and executing a BigQuery query to find the top 5 sub-categories by year-over-year sales growth. I will generate a code block that implements this logic, defining the SQL query with a CTE and pivot, and then using the Python client to run it and display the results.



## Summary:

### Q&A
**Question: What are the top 5 sub-categories with the largest YoY revenue increase, and how were potential data issues handled?**

**Answer:** The top 5 sub-categories with the largest year-over-year (YoY) revenue increase from 2013 to 2014 are:
1.  **Bookcases:** \\$137,170.91 increase.
2.  **Copiers:** \\$134,870.01 increase.
3.  **Appliances:** \\$104,835.65 increase.
4.  **Phones:** \\$98,487.38 increase.
5.  **Storage:** \\$73,951.49 increase.

Two potential data issues were addressed:
*   **Missing Recent Data:** The analysis was adapted to use the years 2013 and 2014, as the dataset is historical and contains no data for 2023-2024.
*   **Missing Sales in One Year:** The query design handles cases where a sub-category might have sales in one year but not the other by treating the missing year's sales as zero, ensuring an accurate calculation of the change.

### Data Analysis Key Findings
*   A query using a Common Table Expression (CTE) was built to calculate the total sales for each sub-category in 2013 and 2014.
*   The data was then pivoted to calculate the absolute revenue change (`yoy_delta`) between the two years for each sub-category.
*   "Bookcases" and "Copiers" showed the most significant growth, with revenue increases of \\$137,170.91 and \\$134,870.01, respectively.
*   The top 5 growing sub-categories are all related to furniture or technology, indicating these were key growth areas for the business in 2014.

### Insights or Next Steps
*   **Insight:** The significant YoY growth in high-value items like "Bookcases" and "Copiers" suggests a successful strategy or favorable market conditions for these products in 2014.
*   **Next Step:** Analyze the profitability of these high-growth sub-categories. A large increase in revenue does not always guarantee a corresponding increase in profit.


## Part E — Window Functions (ROW_NUMBER, RANK, DENSE_RANK, LAG/LEAD, moving averages)
**Aim:** Compare rows across partitions and time; compute trends and ranks without collapsing rows.

### E1. Top product per region (ROW_NUMBER)
**Prompt:**
```
BigQuery SQL only.
Task: For each `Region`, return only the single highest-revenue `Sub_Category`.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `subcat_sales`: SUM(Sales) by `Region`, `Sub_Category`
Add `ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as rn
Final: filter `rn = 1`
Output: `Region`, `Sub_Category`, `total_sales`
Sort by `Region`
```
**Why `ROW_NUMBER` instead of `RANK`?** Ask the model to add a 2-sentence contrast.

In [ ]:
query_string = """
WITH subcat_sales AS (
  SELECT
    region,
    sub_category,
    SUM(sales) AS total_sales,
    ROW_NUMBER() OVER (PARTITION BY region ORDER BY SUM(sales) DESC) AS rn
  FROM
    `mgmt-46700-471119.superstore_data.superstore_clean`
  GROUP BY
    region,
    sub_category
)
SELECT
  region,
  sub_category,
  ROUND(total_sales, 2) as total_sales
FROM
  subcat_sales
WHERE
  rn = 1
ORDER BY
  region;
"""

try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

,region,sub_category,total_sales
0,Africa,Phones,114832.85
1,Canada,Storage,10587.30
2,Caribbean,Copiers,50000.33
3,Central,Phones,370198.04
4,Central Asia,Phones,132714.81
5,EMEA,Phones,114520.61
6,East,Phones,100614.98
7,North,Phones,180421.21
8,North Asia,Bookcases,130066.92
9,Oceania,Chairs,170284.21


### E2. YoY growth with LAG
**Prompt:**
```
BigQuery SQL only.
Task: Compute year-over-year revenue growth for 'Phones' sub-category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Filter to `Sub_Category = 'Phones'`
- Aggregate yearly revenue using EXTRACT(YEAR FROM Order_Date)
- Add `LAG(yearly_revenue) OVER (ORDER BY year)` as `prev_revenue`
- Compute `yoy_pct = 100.0 * (yearly_revenue - prev_revenue) / prev_revenue`
Output: `year`, `yearly_revenue`, `prev_revenue`, `yoy_pct`
Sort by `year` ASC
```
**Ask for**: a guard against divide-by-zero or NULL previous year.

In [ ]:
query_string = """
WITH yearly_phone_sales AS (
  SELECT
    EXTRACT(YEAR FROM order_date) AS year,
    SUM(sales) AS yearly_revenue
  FROM
    `mgmt-46700-471119.superstore_data.superstore_clean`
  WHERE
    sub_category = 'Phones'
  GROUP BY
    year
),
lagged_sales AS (
  SELECT
    year,
    yearly_revenue,
    LAG(yearly_revenue, 1, 0) OVER (ORDER BY year) AS prev_revenue
  FROM
    yearly_phone_sales
)
SELECT
  year,
  ROUND(yearly_revenue, 2) AS yearly_revenue,
  ROUND(prev_revenue, 2) AS prev_revenue,
  ROUND(SAFE_DIVIDE(yearly_revenue - prev_revenue, prev_revenue) * 100, 2) AS yoy_pct
FROM
  lagged_sales
ORDER BY
  year ASC;
"""

try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

,year,yearly_revenue,prev_revenue,yoy_pct
0,2011,337282.48,0.00,NaN
1,2012,364016.27,337282.48,7.93
2,2013,453519.00,364016.27,24.59
3,2014,552006.38,453519.00,21.72


### E3. 3‑month moving average (MA)
**Prompt:**
```
BigQuery SQL only.
Task: For the 'Corporate' segment, compute a 3-month moving average of monthly revenue.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Derive `month` via DATE_TRUNC(Order_Date, MONTH)
- SUM(Sales) per `month`
- Add `AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)` as `ma_3`
Output: `month`, `monthly_revenue`, `ma_3`
Sort by `month` ASC
```
**Tip:** Ask the model to include a 1‑line cost control note (e.g., restrict date range while iterating).

In [ ]:
query_string = """
WITH monthly_corporate_sales AS (
  -- First, calculate monthly revenue just for the 'Corporate' segment
  SELECT
    DATE_TRUNC(order_date, MONTH) AS month,
    SUM(sales) AS monthly_revenue
  FROM
    `mgmt-46700-471119.superstore_data.superstore_clean`
  WHERE
    segment = 'Corporate'
  GROUP BY
    month
)
-- Calculate the 3-month moving average using a window function
SELECT
  month,
  ROUND(monthly_revenue, 2) AS monthly_revenue,
  -- The window function averages the current row and the two preceding rows
  ROUND(AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW), 2) AS ma_3
FROM
  monthly_corporate_sales
ORDER BY
  month ASC
LIMIT 24; -- Cost control note: Limit results during exploration to reduce query cost.
"""

try:
    # Execute the query
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    # Display the results
    display(results_df)

except Exception as e:
    print(f"An error occurred: {e}")

,month,monthly_revenue,ma_3
0,2011-01-01,24279.85,24279.85
1,2011-02-01,42998.44,33639.14
2,2011-03-01,31920.21,33066.16
3,2011-04-01,41015.58,38644.74
4,2011-05-01,54158.02,42364.60
5,2011-06-01,68096.36,54423.32
6,2011-07-01,30188.74,50814.37
7,2011-08-01,69391.86,55892.32
8,2011-09-01,75645.92,58408.84
9,2011-10-01,62717.84,69251.87


In [ ]:
import plotly.express as px

# The results_df from the previous cell contains monthly data for the moving average.
# This chart plots the monthly revenue and its 3-month moving average.
fig = px.line(
    results_df,
    x='month',
    y=['monthly_revenue', 'ma_3'],
    title='Monthly Revenue vs. 3-Month Moving Average for Corporate Segment',
    labels={'month': 'Month', 'value': 'Revenue'}
)

# Update layout for clarity
fig.update_layout(
    xaxis_title='Month',
    yaxis_title='Total Revenue ($)',
    plot_bgcolor='white',
    legend_title_text='Metric'
)

fig.show()

### Insight from Interactivity

The interactivity of the chart reveals the precise magnitude of revenue volatility. For instance, by hovering over the data points, we can pinpoint that the sharpest single-month decline occurred in July 2011, when revenue dropped to \$30,188.74. This was a steep fall from the 3-month moving average, which was \$50,814.37 at that time, highlighting a specific period of significant underperformance that would be difficult to quantify from a static image alone.

## Part F — Debugging & Optimization Prompts
**Aim:** Use the model as a rubber duck for error handling and performance.

### F1. Explain the error, propose a fix
**Prompt:**
```
I ran this BigQuery SQL and got an error:
[PASTE ERROR MESSAGE and the exact SQL here]
Act as a BigQuery trouble‑shooter.
1) Identify the root cause.
2) Propose the smallest possible fix.
3) Suggest a quick sanity check query to verify the fix.
Return only the corrected SQL and a 2‑sentence rationale.
```

### F2. Reduce cost / improve speed
**Prompt:**
```
Act as a BigQuery cost optimizer.
Given this query (below), list 3 ways to reduce scanned bytes and improve performance without changing the business logic.
[PASTE YOUR SQL HERE]
Prioritize: partition filters, column pruning, pre-aggregations, and temporary results via CTEs.
```

## Part G — Validation & Counter‑examples (DIVE: Validate)
**Aim:** Avoid “first‑answer fallacy” by testing alternatives.

### G1. Ask for counter‑queries
**Prompt:**
```
I concluded that 'Tables' is a high‑sales but negative‑profit sub-category due to high discounts.
Create two alternative BigQuery SQL queries that could falsify or nuance this finding:
- One that slices by region and time
- One that controls for order priority or ship mode
Return BigQuery SQL only, then a one-paragraph note on how to compare outcomes.
```

In [ ]:
query_string_regional = """
SELECT
  region,
  EXTRACT(YEAR FROM order_date) AS year,
  ROUND(SUM(sales), 2) AS total_sales,
  ROUND(SUM(profit), 2) AS total_profit,
  ROUND(AVG(discount), 2) AS avg_discount
FROM
  `mgmt-46700-471119.superstore_data.superstore_clean`
WHERE
  sub_category = 'Tables'
GROUP BY
  region,
  year
ORDER BY
  region,
  year;
"""

print("--- Executing Regional & Temporal Analysis for 'Tables' ---")
try:
    # Execute the query
    query_job_regional = client.query(query_string_regional)

    # Wait for the job to complete and get the results as a DataFrame
    results_df_regional = query_job_regional.to_dataframe()

    # Display the results
    print(f"Query successful. Found {len(results_df_regional)} region/year combinations for 'Tables'.")
    display(results_df_regional)

except Exception as e:
    print(f"An error occurred: {e}")

--- Executing Regional & Temporal Analysis for 'Tables' ---
Query successful. Found 49 region/year combinations for 'Tables'.


,region,year,total_sales,total_profit,avg_discount
0,Africa,2011,7682.62,936.76,0.20
1,Africa,2012,1446.07,-1455.32,0.35
2,Africa,2013,7971.18,1980.66,0.00
3,Africa,2014,17432.73,2548.59,0.13
4,Canada,2013,849.36,300.18,0.00
5,Caribbean,2011,443.43,-857.37,0.70
6,Caribbean,2012,6448.62,1300.50,0.23
7,Caribbean,2013,4086.16,-826.42,0.35
8,Caribbean,2014,12683.11,446.69,0.25
9,Central,2011,22667.77,-2507.67,0.34


In [ ]:
query_string_shipping = """
SELECT
  ship_mode,
  ROUND(SUM(sales), 2) AS total_sales,
  ROUND(SUM(profit), 2) AS total_profit,
  ROUND(AVG(discount), 2) AS avg_discount
FROM
  `mgmt-46700-471119.superstore_data.superstore_clean`
WHERE
  sub_category = 'Tables'
GROUP BY
  ship_mode
ORDER BY
  total_profit DESC;
"""

print("--- Executing Logistical Analysis for 'Tables' ---")
try:
    # Execute the query
    query_job_shipping = client.query(query_string_shipping)

    # Wait for the job to complete and get the results as a DataFrame
    results_df_shipping = query_job_shipping.to_dataframe()

    # Display the results
    print(f"Query successful. Found {len(results_df_shipping)} shipping modes for 'Tables'.")
    display(results_df_shipping)

except Exception as e:
    print(f"An error occurred: {e}")

--- Executing Logistical Analysis for 'Tables' ---
Query successful. Found 4 shipping modes for 'Tables'.


,ship_mode,total_sales,total_profit,avg_discount
0,Same Day,21740.39,968.35,0.25
1,First Class,124219.84,-8093.23,0.27
2,Second Class,158446.82,-20877.61,0.31
3,Standard Class,452634.87,-36080.90,0.29


## Part H — Synthesis (DIVE: Extend)
**Aim:** Turn analysis into business‑ready insights.

### H1. Executive‑style summary
**Prompt:**
```
Act as a business strategist.
Based on the following metrics/figures (briefly summarize your results here), write a 4-sentence executive summary:
- 1 sentence: what changed and by how much
- 1 sentence: why it likely changed (drivers)
- 1 sentence: recommended action (who/what/when)
- 1 sentence: metric to monitor next
```

### H2. Convert final SQL into an automated job (optional)
**Prompt (use only after your SQL is final):**
```
Convert my final BigQuery SQL into a Python script that can run as a scheduled job from Colab or Cloud Functions.
Requirements:
- Use python‑bigquery client
- Parameterize date range
- Write results to a destination table `[YOUR_PROJECT].analytics.outputs_kpi`
- Add basic error handling & logging
Return one complete runnable script.
```

---
## Submission checklist
- [ ] Kept prompts precise and reproducible  
- [ ] Captured at least **one** CTE query and **one** window function query  
- [ ] Documented **two** validation attempts (counter‑queries or alternate slice)  
- [ ] Wrote a 4‑sentence executive summary based on results  
- [ ] (Optional) Converted final query into a scheduled job
---

# Task
Using the BigQuery table `\"mgmt-46700-471119.superstore_data.superstore_clean\"`, first attempt to find the top 3 best-selling product sub-categories for the year 2023 to demonstrate that the query returns no data due to the historical nature of the dataset. Then, modify the query to find the top 3 best-selling sub-categories for 2014, the last full year of data available. Finally, deliver the result in a text cell with the statement: “The top 3 sub-categories in 2014 were [Sub-category 1], [Sub-category 2], and [Sub-category 3].”

## Generate and Execute 2023 Query

### Subtask:
First, I will generate and execute the BigQuery SQL query exactly as specified in your prompt, targeting the year 2023. This step will demonstrate that no data is returned because the dataset is historical.


**Reasoning**:
I will create a Python code block to execute the BigQuery query for 2023. This query is expected to return an empty DataFrame, as the dataset is historical and does not contain data for that year. This step will fulfill the first part of the plan, demonstrating the need to adjust the query to a relevant year.



In [ ]:
# This query attempts to find top-selling sub-categories for 2023.
# It is expected to return no results because the dataset is historical.
query_string_2023 = """
WITH yearly_sales AS (
  -- In a CTE, calculate total sales per sub-category, filtered for 2023
  SELECT
    sub_category,
    SUM(sales) AS total_sales
  FROM
    `mgmt-46700-471119.superstore_data.superstore_clean`
  WHERE
    EXTRACT(YEAR FROM order_date) = 2023
  GROUP BY
    sub_category
)
-- Select the top 3 sub-categories from the filtered data
SELECT
  sub_category,
  total_sales
FROM
  yearly_sales
ORDER BY
  total_sales DESC
LIMIT 3;
"""

print("--- Attempting to Query for Top 3 Sub-Categories in 2023 ---")
try:
    # Execute the query
    query_job_2023 = client.query(query_string_2023)

    # Wait for the job to complete and get the results as a DataFrame
    results_df_2023 = query_job_2023.to_dataframe()

    # Display the results
    if results_df_2023.empty:
        print("Query successful. As expected, no data was found for the year 2023.")
        display(results_df_2023)
    else:
        print(f"Query successful. Found {len(results_df_2023)} results for 2023.")
        display(results_df_2023)

except Exception as e:
    print(f"An error occurred: {e}")

--- Attempting to Query for Top 3 Sub-Categories in 2023 ---
Query successful. As expected, no data was found for the year 2023.


,sub_category,total_sales


# Task
Using the BigQuery table `"mgmt-46700-471119.superstore_data.superstore_clean"`, generate and execute a query with a Common Table Expression (CTE) to find the top 3 product sub-categories by total sales for the year 2023. Based on the expected empty result, deliver a final summary in a text cell stating that no top-selling sub-categories were found for 2023.

## Generate and Execute 2023 Query

### Subtask:
Generate and execute a BigQuery query to find the top 3 selling sub-categories for 2023.


**Reasoning**:
The user has provided detailed instructions for creating and executing a BigQuery query. I will generate a code block that implements all these steps, including defining the SQL string with a CTE, executing it within a `try...except` block, and checking the resulting DataFrame to confirm that it is empty, as expected.



In [ ]:
# This query attempts to find top-selling sub-categories for 2023.
# It is expected to return no results because the dataset is historical.
query_string_2023 = """
WITH yearly_sales AS (
  -- In a CTE, calculate total sales per sub-category, filtered for 2023
  SELECT
    sub_category,
    SUM(sales) AS total_sales
  FROM
    `mgmt-46700-471119.superstore_data.superstore_clean`
  WHERE
    EXTRACT(YEAR FROM order_date) = 2023
  GROUP BY
    sub_category
)
-- Select the top 3 sub-categories from the filtered data
SELECT
  sub_category,
  total_sales
FROM
  yearly_sales
ORDER BY
  total_sales DESC
LIMIT 3;
"""

print("--- Attempting to Query for Top 3 Sub-Categories in 2023 ---")
try:
    # Execute the query
    query_job_2023 = client.query(query_string_2023)

    # Wait for the job to complete and get the results as a DataFrame
    results_df_2023 = query_job_2023.to_dataframe()

    # Display the results
    if results_df_2023.empty:
        print("Query successful. As expected, no data was found for the year 2023.")
        display(results_df_2023)
    else:
        print(f"Query successful. Found {len(results_df_2023)} results for 2023.")
        display(results_df_2023)

except Exception as e:
    print(f"An error occurred: {e}")

--- Attempting to Query for Top 3 Sub-Categories in 2023 ---
Query successful. As expected, no data was found for the year 2023.


,sub_category,total_sales
